In [1]:
import requests, json, praw, time, csv, os
from ratelimit import limits, sleep_and_retry
from itertools import islice

# =============================
# 1. Constants and Credentials
# =============================

REDDIT_CLIENT_ID = "7XS2VDcdCA5iXt1n90nc7w"
REDDIT_CLIENT_SECRET = 'lgAmuWHjL5j7IofUCJQtU4xBQfEicQ'
REDDIT_USER_AGENT = 'review_scraper/0.1 by u/Advanced-Ad-5151'
OPENROUTER_API_KEY = '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'

REDDIT_RATE_LIMIT_CALLS = 60
REDDIT_RATE_LIMIT_PERIOD = 60  # seconds

BATCH_SIZE = 5
OUTPUT_CSV = "labeled_comments.csv"


POST_LIMIT = None

In [2]:
# =============================
# 2. Set Up Reddit API Connection
# =============================
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# =============================
# 3. Rate-Limited Reddit Call
# =============================
@sleep_and_retry
@limits(calls=REDDIT_RATE_LIMIT_CALLS, period=REDDIT_RATE_LIMIT_PERIOD)
def fetch_submission_comments(submission):
    submission.comments.replace_more(limit=0)
    return [comment.body for comment in submission.comments]

# =============================
# 4. Extract Comments from Subreddit
# =============================
def fetch_comments_from_subreddit(subreddit_name, post_limit=POST_LIMIT):
    subreddit = reddit.subreddit(subreddit_name)
    all_comments = []

    for submission in subreddit.hot(limit=post_limit):
        try:
            comments = fetch_submission_comments(submission)
            all_comments.extend(comments)
        except Exception as e:
            print(f"Error fetching comments from {submission.title}: {e}")

    return all_comments

# =============================
# 5. Save Functions
# =============================
def save_comments_to_txt(comments, path):
    with open(os.path.join(path, "reddit_comments.txt"), "w", encoding="utf-8") as f:
        for comment in comments:
            f.write(comment + "\n")

def save_comments_to_csv(comments, path):
    with open(os.path.join(path, "reddit_comments.csv"), "w", encoding="utf-8", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["comment"])
        for comment in comments:
            writer.writerow([comment])

def save_comments_to_json(comments, path):
    with open(os.path.join(path, "reddit_comments.json"), "w", encoding="utf-8") as f:
        json.dump(comments, f, ensure_ascii=False, indent=2)

# =============================
# 6. Run Script
# =============================
subreddits = []

for sub_name in subreddits:
    print(f"Fetching...Subreddit: r/{sub_name}")
    try:
        comments = fetch_comments_from_subreddit(sub_name)
    
        folder_name = f"r-{sub_name}"
        os.makedirs(folder_name, exist_ok=True)
    
        # Save to files inside folders, respectively
        print(f"Saving files in {folder_name}...")
        save_comments_to_txt(comments, folder_name)
        save_comments_to_csv(comments, folder_name)
        save_comments_to_json(comments, folder_name)
    
        print(f"Fetched and saved {len(comments)} comments from r/{sub_name}\n")

    except Exception as e:
        print(f"Exception occured: {e}")
        print(f"Skipped {sub_name}...\n")

Fetching...Subreddit: r/madlads
Saving files in r-madlads...
Fetched and saved 20594 comments from r/madlads

Fetching...Subreddit: r/bidenbro
Exception occured: received 403 HTTP response
Skipped bidenbro...

Fetching...Subreddit: r/memeeconomy
Saving files in r-memeeconomy...
Fetched and saved 912 comments from r/memeeconomy

Fetching...Subreddit: r/rarepuppers
Saving files in r-rarepuppers...
Fetched and saved 11500 comments from r/rarepuppers

Fetching...Subreddit: r/wholesomememes
Saving files in r-wholesomememes...
Fetched and saved 10881 comments from r/wholesomememes

Fetching...Subreddit: r/dankchristianmemes
Saving files in r-dankchristianmemes...
Fetched and saved 6812 comments from r/dankchristianmemes

Fetching...Subreddit: r/terriblefacebookmemes
Saving files in r-terriblefacebookmemes...
Fetched and saved 23860 comments from r/terriblefacebookmemes

Fetching...Subreddit: r/dank_meme
Saving files in r-dank_meme...
Fetched and saved 3353 comments from r/dank_meme

Fetching

---

---

---